In [1]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import cross_validation

import numpy
import math

In [32]:
train_df = DataFrame.from_csv("train.csv", index_col=['Date', 'Store'], parse_dates=['Date'])
del train_df['Sales']
train_df.head()

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


DayOfWeek  Customers  Open  Promo StateHoliday  \
Date       Store                                                   
2015-07-31 1              5        555     1      1            0   
           2              5        625     1      1            0   
           3              5        821     1      1            0   
           4              5       1498     1      1            0   
           5              5        559     1      1            0   

                  SchoolHoliday  
Date       Store                 
2015-07-31 1                  1  
           2                  1  
           3                  1  
           4                  1  
           5                  1

In [33]:
df_basic = DataFrame.from_csv("train_features_basic.csv", index_col=['Date', 'Store'])
train_df = train_df[['Customers']].join(df_basic, how='outer')

In [34]:
train_df.head()

Customers  DayOfWeek  Open  Promo  StateHoliday  \
Date       Store                                                    
2015-07-31 1            555          5     1      1             0   
           2            625          5     1      1             0   
           3            821          5     1      1             0   
           4           1498          5     1      1             0   
           5            559          5     1      1             0   

                  SchoolHoliday  cal:dayOfMonth  cal:dayOfWeek  cal:dayofyear  \
Date       Store                                                                
2015-07-31 1                  1              31              4            212   
           2                  1              31              4            212   
           3                  1              31              4            212   
           4                  1              31              4            212   
           5                  1              31              4            212   

                  cal:month  cal:quarter  cal:weekofyear  cal:year  
Date       Store                                                    
2015-07-31 1              7            3              31      2015  
           2              7            3              31      2015  
           3              7            3              31      2015  
           4              7            3              31      2015  
           5              7            3              31      2015

# "Train" the model: Calculate mean per group over full training set

In [4]:
means_df = train_df[train_df['Customers'] > 0]
means_df['Open'] = train_df['Open'].fillna(1)


/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Customers'].mean()
    means = means.reset_index()
    return means

means_df = pandas.merge(train_df, calculate_means(means_df), on = ['Store','DayOfWeek','Promo'], how='left', suffixes=["", "_predicted"])
means_df['Customers_predicted'] = means_df['Customers_predicted'].fillna(means_df['Customers_predicted'].mean())


In [6]:
means_df.head()

,Store,Date,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday,Customers_predicted
0,1,2015-07-31,5,555,1,1,0,1,588.194030
1,2,2015-07-31,5,625,1,1,0,1,600.441176
2,3,2015-07-31,5,821,1,1,0,1,838.223881
3,4,2015-07-31,5,1498,1,1,0,1,1373.895522
4,5,2015-07-31,5,559,1,1,0,1,598.462687


### Score the model

In [7]:
def score(predicted, actual): # RootMeanSquaredPercentError RMSPE
    score_df = DataFrame.from_dict({"Predicted": predicted, "Actual": actual})
    score_df = score_df[score_df['Actual'] != 0]
    score_df['PercentError'] =  (score_df['Actual'] - score_df['Predicted']) / score_df['Actual']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

In [8]:
score_df = means_df[['Customers', 'Customers_predicted']].copy()
score_df = score_df[score_df['Customers'] != 0]
score_df['PercentError'] =  (score_df['Customers'] - score_df['Customers_predicted']) / score_df['Customers']
score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
math.sqrt(score_df['PercentErrorSquared'].mean())

0.5216687926418426

## Adding basic features

In [9]:
means_df.set_index(['Date', 'Store'], inplace=True)

In [10]:
means_df = means_df[['Customers', 'Customers_predicted']]
means_df.columns = ['Customers', 'Customers-predicted_means']
df_basic = DataFrame.from_csv("train_features_basic.csv", index_col=['Date', 'Store'])

In [11]:
df_features = means_df.join(df_basic, how='outer')

In [12]:
df_features['Customers-predicted_means'] = df_features['Customers-predicted_means'].fillna(df_features['Customers-predicted_means'].mean())

In [13]:
df_features.head()

Customers  Customers-predicted_means  DayOfWeek  Open  \
Date       Store                                                          
2015-07-31 1            555                 588.194030          5     1   
           2            625                 600.441176          5     1   
           3            821                 838.223881          5     1   
           4           1498                1373.895522          5     1   
           5            559                 598.462687          5     1   

                  Promo  StateHoliday  SchoolHoliday  cal:dayOfMonth  \
Date       Store                                                       
2015-07-31 1          1             0              1              31   
           2          1             0              1              31   
           3          1             0              1              31   
           4          1             0              1              31   
           5          1             0              1              31   

                  cal:dayOfWeek  cal:dayofyear  cal:month  cal:quarter  \
Date       Store                                                         
2015-07-31 1                  4            212          7            3   
           2                  4            212          7            3   
           3                  4            212          7            3   
           4                  4            212          7            3   
           5                  4            212          7            3   

                  cal:weekofyear  cal:year  
Date       Store                            
2015-07-31 1                  31      2015  
           2                  31      2015  
           3                  31      2015  
           4                  31      2015  
           5                  31      2015

In [14]:
def score(predicted, actual): # RootMeanSquaredPercentError RMSPE
    score_df = DataFrame.from_dict({"Predicted": predicted, "Actual": actual})
    score_df = score_df[score_df['Actual'] != 0]
    score_df['PercentError'] =  (score_df['Actual'] - score_df['Predicted']) / score_df['Actual']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

In [15]:
df_features.reset_index(inplace=True)
del df_features['Date']

In [16]:
regressor_big = ExtraTreesRegressor(n_estimators=50, n_jobs=-1)
features = [col for col in df_features.columns if col != 'Sales' and col != 'Customers']
predictions = cross_validation.cross_val_predict(regressor_big, df_features[features], df_features['Customers'])
                                   

In [17]:
score(predicted=predictions, actual=df_features['Customers'])

0.5385941010102885

In [18]:
train_df

,Store,Date,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2015-07-31,5,555,1,1,0,1
1,2,2015-07-31,5,625,1,1,0,1
2,3,2015-07-31,5,821,1,1,0,1
3,4,2015-07-31,5,1498,1,1,0,1
4,5,2015-07-31,5,559,1,1,0,1
5,6,2015-07-31,5,589,1,1,0,1
6,7,2015-07-31,5,1414,1,1,0,1
7,8,2015-07-31,5,833,1,1,0,1
8,9,2015-07-31,5,687,1,1,0,1
9,10,2015-07-31,5,681,1,1,0,1


In [19]:
score(predicted=df_features['Customers-predicted_means'][0:10], actual=train_df['Customers'][0:10])

0.13465738455512313

In [20]:
predictions

array([ 603.  ,  607.22,  883.4 , ...,    0.  ,    0.  ,    0.  ])

# Save model


In [21]:
train_df.set_index(['Date', 'Store'], inplace=True)
test_df.set_index(['Date', 'Store'], inplace=True)


In [22]:
train_df['Customers-prediction_mean_error'] = (train_df['Customers_predicted'] - train_df['Customers']) / train_df['Customers']
train_df['Customers-prediction_mean_error'] = train_df['Customers-prediction_mean_error'].fillna(0) # when sales is 0, sales_predicted is nan.

KeyError: 'Customers_predicted'

In [ ]:
train_df[['Customers_predicted', 'Customers-prediction_mean_error']].to_csv("train-features-predicted_mean-customers.csv")

In [ ]:
test_df.reset_index(inplace=True)

In [ ]:

test_df = pandas.merge(test_df, means, on = ['Store','DayOfWeek','Promo'], how='left')

In [ ]:
test_df.head()

In [ ]:
test_df['Customers_predicted'] = test_df['Customers']
del test_df['Customers']

In [ ]:
test_df[['Customers_predicted']].to_csv("test-features-predicted_mean-customers.csv")